https://machinelearningmastery.com/practical-guide-to-gan-failure-modes/

In [28]:
# example of training a stable gan for generating a handwritten digit
from os import makedirs
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.initializers import RandomNormal
from matplotlib import pyplot
import os

In [29]:
FILE_PATH = os.path.dirname(os.path.abspath("__file__"))
print(FILE_PATH)


/home/thien/Code/Torch_TensorFlow_tutor/Torch_more_Advanced/GANs_tutorial/Web2_TF


In [30]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# downsample to 14x14
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 7x7
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model



In [31]:
# define the standalone generator model
def define_generator(latent_dim):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model



In [32]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model



In [33]:
# load mnist images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# select all of the examples for a given class
	selected_ix = trainy == 8
	X = X[selected_ix]
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X



In [34]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y



In [35]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input



In [36]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y



In [37]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	pyplot.savefig(FILE_PATH+'/results_collapse/generated_plot_%03d.png' % (step+1))
	pyplot.close()
	# save the generator model
	g_model.save(FILE_PATH+'/results_collapse/model_%03d.h5' % (step+1))



In [38]:
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
	# plot loss
	pyplot.subplot(2, 1, 1)
	pyplot.plot(d1_hist, label='d-real')
	pyplot.plot(d2_hist, label='d-fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	# plot discriminator accuracy
	pyplot.subplot(2, 1, 2)
	pyplot.plot(a1_hist, label='acc-real')
	pyplot.plot(a2_hist, label='acc-fake')
	pyplot.legend()
	# save plot to file
	pyplot.savefig(FILE_PATH+'/results_collapse/plot_line_plot_loss.png')
	pyplot.close()



In [39]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
	# calculate the number of batches per epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the total iterations based on batch and epoch
	n_steps = bat_per_epo * n_epochs
	# calculate the number of samples in half a batch
	half_batch = int(n_batch / 2)
	# prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
			(i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		a1_hist.append(d_acc1)
		a2_hist.append(d_acc2)
		# evaluate the model performance every 'epoch'
		if (i+1) % bat_per_epo == 0:
			summarize_performance(i, g_model, latent_dim)
	plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)



In [40]:
# make folder for results
makedirs(FILE_PATH+'/results_collapse', exist_ok=True)
# size of the latent space
latent_dim = 1
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
print(dataset.shape)

(5851, 28, 28, 1)


In [42]:
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

2/2 [==============================] - 0s 4ms/step
>1, d1=0.327, d2=0.213 g=3.117, a1=82, a2=100
2/2 [==============================] - 0s 3ms/step
>2, d1=0.268, d2=0.220 g=3.159, a1=90, a2=100
2/2 [==============================] - 0s 3ms/step
>3, d1=0.408, d2=0.248 g=2.851, a1=85, a2=93
2/2 [==============================] - 0s 2ms/step
>4, d1=0.385, d2=0.196 g=2.998, a1=85, a2=98
2/2 [==============================] - 0s 3ms/step
>5, d1=0.257, d2=0.183 g=3.197, a1=90, a2=100
2/2 [==============================] - 0s 3ms/step
>6, d1=0.378, d2=0.264 g=3.011, a1=85, a2=96
2/2 [==============================] - 0s 3ms/step
>7, d1=0.292, d2=0.252 g=3.103, a1=90, a2=92
2/2 [==============================] - 0s 4ms/step
>8, d1=0.379, d2=0.141 g=2.897, a1=84, a2=100
2/2 [==============================] - 0s 3ms/step
>9, d1=0.244, d2=0.171 g=2.679, a1=89, a2=100
2/2 [==============================] - 0s 3ms/step
>10, d1=0.120, d2=0.126 g=2.767, a1=96, a2=100
2/2 [============================

2/2 [==============================] - 0s 3ms/step
>46, d1=0.169, d2=0.122 g=3.396, a1=95, a2=100
2/2 [==============================] - 0s 3ms/step
>47, d1=0.286, d2=0.415 g=4.253, a1=87, a2=76
2/2 [==============================] - 0s 3ms/step
>48, d1=0.349, d2=0.100 g=4.080, a1=87, a2=100
2/2 [==============================] - 0s 3ms/step
>49, d1=0.405, d2=0.331 g=3.638, a1=84, a2=90
2/2 [==============================] - 0s 4ms/step
>50, d1=0.210, d2=0.163 g=3.614, a1=89, a2=100
2/2 [==============================] - 0s 3ms/step
>51, d1=0.372, d2=0.299 g=3.270, a1=82, a2=98
2/2 [==============================] - 0s 3ms/step
>52, d1=0.383, d2=0.281 g=3.477, a1=82, a2=95
2/2 [==============================] - 0s 4ms/step
>53, d1=0.431, d2=0.331 g=3.402, a1=81, a2=76
2/2 [==============================] - 0s 3ms/step
>54, d1=0.367, d2=0.130 g=3.039, a1=87, a2=100
2/2 [==============================] - 0s 3ms/step
>55, d1=0.375, d2=0.338 g=3.215, a1=82, a2=95
2/2 [=====================

2/2 [==============================] - 0s 3ms/step
>91, d1=0.311, d2=0.555 g=4.535, a1=82, a2=59
2/2 [==============================] - 0s 3ms/step
>92, d1=0.869, d2=0.224 g=3.569, a1=68, a2=100
2/2 [==============================] - 0s 3ms/step
>93, d1=0.400, d2=0.534 g=4.293, a1=82, a2=73
2/2 [==============================] - 0s 3ms/step
>94, d1=0.664, d2=0.187 g=3.592, a1=71, a2=95
2/2 [==============================] - 0s 2ms/step
>95, d1=0.327, d2=0.363 g=3.956, a1=81, a2=84
2/2 [==============================] - 0s 2ms/step
>96, d1=0.258, d2=0.225 g=4.345, a1=89, a2=79
2/2 [==============================] - 0s 2ms/step
>97, d1=0.416, d2=0.317 g=4.369, a1=78, a2=70
2/2 [==============================] - 0s 3ms/step
>98, d1=0.458, d2=0.146 g=3.386, a1=87, a2=100
2/2 [==============================] - 0s 2ms/step
>99, d1=0.134, d2=0.197 g=3.924, a1=93, a2=100
2/2 [==============================] - 0s 3ms/step
>100, d1=0.388, d2=0.105 g=3.284, a1=84, a2=100
2/2 [====================

2/2 [==============================] - 0s 3ms/step
>136, d1=0.459, d2=0.481 g=4.938, a1=73, a2=82
2/2 [==============================] - 0s 3ms/step
>137, d1=0.564, d2=0.143 g=3.824, a1=76, a2=100
2/2 [==============================] - 0s 3ms/step
>138, d1=0.514, d2=0.765 g=5.315, a1=75, a2=57
2/2 [==============================] - 0s 2ms/step
>139, d1=0.530, d2=0.040 g=4.120, a1=73, a2=100
2/2 [==============================] - 0s 3ms/step
>140, d1=0.395, d2=0.467 g=4.028, a1=82, a2=87
2/2 [==============================] - 0s 3ms/step
>141, d1=0.401, d2=0.108 g=3.227, a1=81, a2=100
2/2 [==============================] - 0s 3ms/step
>142, d1=0.169, d2=0.596 g=4.926, a1=95, a2=76
2/2 [==============================] - 0s 3ms/step
>143, d1=0.706, d2=0.523 g=5.288, a1=71, a2=76
2/2 [==============================] - 0s 3ms/step
>144, d1=1.226, d2=0.319 g=4.166, a1=37, a2=96
2/2 [==============================] - 0s 3ms/step
>145, d1=0.682, d2=0.511 g=4.955, a1=70, a2=73
2/2 [============

2/2 [==============================] - 0s 3ms/step
>181, d1=0.465, d2=0.127 g=3.363, a1=75, a2=100
2/2 [==============================] - 0s 2ms/step
>182, d1=0.233, d2=0.457 g=4.020, a1=89, a2=76
2/2 [==============================] - 0s 3ms/step
>183, d1=0.489, d2=0.156 g=3.514, a1=73, a2=100
2/2 [==============================] - 0s 3ms/step
>184, d1=0.412, d2=0.470 g=3.766, a1=78, a2=81
2/2 [==============================] - 0s 4ms/step
>185, d1=0.400, d2=0.184 g=3.285, a1=78, a2=100
2/2 [==============================] - 0s 2ms/step
>186, d1=0.299, d2=0.359 g=3.340, a1=90, a2=100
2/2 [==============================] - 0s 3ms/step
>187, d1=0.483, d2=0.287 g=3.315, a1=79, a2=100
2/2 [==============================] - 0s 3ms/step
>188, d1=0.525, d2=0.433 g=3.149, a1=76, a2=76
2/2 [==============================] - 0s 3ms/step
>189, d1=0.488, d2=0.366 g=3.084, a1=81, a2=82
2/2 [==============================] - 0s 3ms/step
>190, d1=0.544, d2=0.458 g=3.709, a1=78, a2=70
2/2 [==========

2/2 [==============================] - 0s 3ms/step
>226, d1=0.907, d2=0.419 g=2.731, a1=54, a2=79
2/2 [==============================] - 0s 3ms/step
>227, d1=0.468, d2=0.450 g=2.727, a1=82, a2=85
2/2 [==============================] - 0s 4ms/step
>228, d1=0.602, d2=0.656 g=2.124, a1=78, a2=71
2/2 [==============================] - 0s 3ms/step
>229, d1=0.643, d2=0.508 g=2.102, a1=73, a2=81
2/2 [==============================] - 0s 3ms/step
>230, d1=0.826, d2=0.405 g=2.126, a1=73, a2=100
2/2 [==============================] - 0s 4ms/step
>231, d1=0.578, d2=0.488 g=2.095, a1=78, a2=78
2/2 [==============================] - 0s 3ms/step
>232, d1=0.322, d2=0.914 g=2.095, a1=90, a2=29
2/2 [==============================] - 0s 3ms/step
>233, d1=0.860, d2=1.186 g=2.052, a1=59, a2=21
2/2 [==============================] - 0s 4ms/step
>234, d1=0.886, d2=0.699 g=2.348, a1=54, a2=28
2/2 [==============================] - 0s 3ms/step
>235, d1=1.415, d2=0.730 g=2.001, a1=32, a2=43
2/2 [==============

2/2 [==============================] - 0s 3ms/step
>271, d1=0.382, d2=0.364 g=1.984, a1=79, a2=100
2/2 [==============================] - 0s 3ms/step
>272, d1=0.437, d2=0.459 g=1.964, a1=78, a2=100
2/2 [==============================] - 0s 3ms/step
>273, d1=0.387, d2=0.339 g=2.098, a1=81, a2=100
2/2 [==============================] - 0s 3ms/step
>274, d1=0.396, d2=0.327 g=2.097, a1=79, a2=100
2/2 [==============================] - 0s 3ms/step
>275, d1=0.573, d2=0.485 g=1.912, a1=75, a2=87
2/2 [==============================] - 0s 4ms/step
>276, d1=0.492, d2=0.520 g=2.045, a1=79, a2=92
2/2 [==============================] - 0s 2ms/step
>277, d1=0.566, d2=0.497 g=1.929, a1=71, a2=90
2/2 [==============================] - 0s 3ms/step
>278, d1=0.671, d2=0.706 g=1.899, a1=64, a2=56
2/2 [==============================] - 0s 3ms/step
>279, d1=0.931, d2=0.800 g=1.693, a1=54, a2=14
2/2 [==============================] - 0s 3ms/step
>280, d1=0.698, d2=0.739 g=1.764, a1=62, a2=42
2/2 [===========

2/2 [==============================] - 0s 3ms/step
>316, d1=0.731, d2=0.222 g=2.640, a1=59, a2=100
2/2 [==============================] - 0s 2ms/step
>317, d1=0.556, d2=0.764 g=2.015, a1=71, a2=37
2/2 [==============================] - 0s 3ms/step
>318, d1=0.625, d2=1.484 g=2.728, a1=78, a2=15
2/2 [==============================] - 0s 3ms/step
>319, d1=0.981, d2=0.141 g=3.170, a1=54, a2=100
2/2 [==============================] - 0s 3ms/step
>320, d1=0.721, d2=0.278 g=2.751, a1=60, a2=98
2/2 [==============================] - 0s 3ms/step
>321, d1=0.473, d2=0.528 g=3.087, a1=84, a2=65
2/2 [==============================] - 0s 3ms/step
>322, d1=0.413, d2=0.328 g=2.748, a1=85, a2=89
2/2 [==============================] - 0s 2ms/step
>323, d1=0.497, d2=0.491 g=2.201, a1=78, a2=82
2/2 [==============================] - 0s 3ms/step
>324, d1=0.544, d2=0.668 g=1.747, a1=73, a2=78
2/2 [==============================] - 0s 3ms/step
>325, d1=0.599, d2=1.208 g=1.718, a1=71, a2=51
2/2 [=============

2/2 [==============================] - 0s 3ms/step
>361, d1=0.730, d2=0.246 g=2.165, a1=56, a2=100
2/2 [==============================] - 0s 3ms/step
>362, d1=0.459, d2=0.420 g=1.904, a1=84, a2=82
2/2 [==============================] - 0s 2ms/step
>363, d1=0.464, d2=0.529 g=1.856, a1=78, a2=71
2/2 [==============================] - 0s 2ms/step
>364, d1=0.592, d2=0.445 g=1.825, a1=79, a2=84
2/2 [==============================] - 0s 3ms/step
>365, d1=0.474, d2=0.357 g=1.890, a1=75, a2=100
2/2 [==============================] - 0s 3ms/step
>366, d1=0.605, d2=0.363 g=1.823, a1=70, a2=100
2/2 [==============================] - 0s 3ms/step
>367, d1=0.430, d2=0.367 g=1.861, a1=79, a2=100
2/2 [==============================] - 0s 3ms/step
>368, d1=0.494, d2=0.387 g=1.816, a1=75, a2=100
2/2 [==============================] - 0s 3ms/step
>369, d1=0.323, d2=0.438 g=1.883, a1=87, a2=100
2/2 [==============================] - 0s 3ms/step
>370, d1=0.480, d2=0.448 g=1.878, a1=73, a2=81
2/2 [=========

2/2 [==============================] - 0s 3ms/step
>406, d1=0.657, d2=0.530 g=2.338, a1=70, a2=65
2/2 [==============================] - 0s 2ms/step
>407, d1=0.771, d2=0.721 g=1.983, a1=59, a2=59
2/2 [==============================] - 0s 3ms/step
>408, d1=0.725, d2=0.980 g=2.045, a1=62, a2=23
2/2 [==============================] - 0s 3ms/step
>409, d1=1.022, d2=0.757 g=1.782, a1=42, a2=31
2/2 [==============================] - 0s 3ms/step
>410, d1=1.014, d2=0.770 g=1.620, a1=42, a2=39
2/2 [==============================] - 0s 3ms/step
>411, d1=1.034, d2=0.707 g=1.606, a1=45, a2=50
2/2 [==============================] - 0s 2ms/step
>412, d1=0.832, d2=0.536 g=1.556, a1=56, a2=70
2/2 [==============================] - 0s 3ms/step
>413, d1=0.634, d2=0.569 g=1.690, a1=64, a2=59
2/2 [==============================] - 0s 3ms/step
>414, d1=0.487, d2=0.483 g=1.829, a1=76, a2=100
2/2 [==============================] - 0s 3ms/step
>415, d1=0.581, d2=0.384 g=1.735, a1=70, a2=100
2/2 [=============